In [26]:
import torch
print(torch.__version__)

1.5.0+cu101


In [27]:
%cd ./drive/My\ Drive/

[Errno 2] No such file or directory: './drive/My Drive/'
/content/drive/My Drive


/content/drive/My Drive


In [0]:
file = open('corpus.txt')

In [0]:
lis = file.readlines()


In [30]:
print(len(lis))

30719


In [31]:
for i in range(10):
  print(lis[i])

cystic fibrosis affects 30000 children young adults us alone

inhaling mists salt water reduce pus infection fills airways cystic fibrosis sufferers although side effects include nasty coughing fit harsh taste

thats conclusion two studies published weeks issue new england journal medicine

found inhaling mist salt content 7 9 improved lung function cases produced less absenteeism school work

cystic fibrosis progressive frequently fatal genetic disease affects 30000 young adults children us alone marked thickening mucus makes harder clear lungs debris bacteria

salt water solution really opens new avenue approaching patients cystic fibrosis treat says dr gail weinmann us national heart lung blood institute sponsored one studies

mark elkins royal prince alfred hospital sydney australia colleagues authored one new published studies

team found 83 volunteers regularly inhaled 7 mist salty water fewer breathing problems less absenteeism school work inhaled solution salt content 1

adding

In [0]:
unique_words = []
for i in range(len(lis)):
  sentence = lis[i]
  words = sentence.split()
  for word in words:
    if(word not in unique_words):
      unique_words.append(word)

# Get all unique words

In [33]:
print(len(unique_words))

30477


In [0]:
mapping = {}
reverse_mapping = {}
for index,word in enumerate(unique_words):
  mapping[word] = index
  reverse_mapping[index] = word
 
# Make indexes for unique words

In [0]:
import pickle
pickle.dump(mapping,open('Mapping.pkl','wb'))
pickle.dump(reverse_mapping,open('Reverse_Mapping.pkl','wb'))

In [36]:
context_dict = {}
outer_count = 0
for sentence in lis:
  words = sentence.split()
  for i in range(len(words)):
    word = words[i]
    counter=0
    if(word not in context_dict):
      context_dict[word] = [] 
    if(i-1>=0):
      context_dict[word].append(words[i-1])
      counter+=1
    if(i-2>=0):
      context_dict[word].append(words[i-2])
      counter+=1
    if(i+1<len(words)):
      context_dict[word].append(words[i+1]) 
      counter+=1
    if(i+2<len(words)):
      context_dict[word].append(words[i+2]) 
      counter+=1
    if(counter!=4):
      outer_count+=1
print(outer_count)








120915


In [0]:
data_pairs = []
import random
for i in context_dict:
  for j in context_dict[i]:
    data_pairs.append([mapping[i],mapping[j],1])


In [0]:
frequency_dictionary = {}
for i in data_pairs:
  if(str(i) not in frequency_dictionary):
    frequency_dictionary[str(i)] = 0
  frequency_dictionary[str(i)]+=1


In [39]:
removed_data_pairs = []
for i in data_pairs:
  if(frequency_dictionary[str(i)]!=1):
    removed_data_pairs.append(i)
print(len(removed_data_pairs))

520814


In [40]:
count = 0
for i in context_dict:
  for j in context_dict[i]:
    count+=1
print(count)

1379180


In [0]:
import torch.nn as nn
import torch.nn.functional as F
class Skip_Gram_W2V(nn.Module):
  
  def __init__(self,vocabulary_size,vector_size):
    super(Skip_Gram_W2V,self).__init__()
    self.hidden_embedding = nn.Embedding(vocabulary_size,vector_size) # This layer contains all the word vectors by the end of the training
  
  def forward(self,center_word_index,context_word_index):
    center_word_tensor = torch.tensor(center_word_index,dtype=torch.long)
    context_word_tensor = torch.tensor(context_word_index,dtype=torch.long)
    center_hidden_embedding = self.hidden_embedding(center_word_tensor)
    context_hidden_embedding = self.hidden_embedding(context_word_tensor) 
    prod_result = torch.matmul(center_hidden_embedding,torch.t(context_hidden_embedding)) 
    return F.logsigmoid(prod_result)


  




In [47]:
import torch.optim as optim
import random
import torch
number_of_epochs = 2
criterion = nn.MSELoss()
vocab_size = len(unique_words)
model = Skip_Gram_W2V(vocab_size,60)
model.load_state_dict(torch.load('5000000layer.pth'))
optimizer = optim.SGD(list(model.parameters()),lr=0.001)

for i in range(4):
  total_criterion_loss = 0
  count=0
  for data_pair in removed_data_pairs:
    center = data_pair[0]
    context = data_pair[1] 
    ground_truth = data_pair[2]

    ground_truth_tensor = torch.tensor([ground_truth],dtype=torch.long)
    inter_result = model([center],[context])
    inter_criterion_loss = criterion(ground_truth_tensor,inter_result[0])
    total_criterion_loss+=inter_criterion_loss.data

    optimizer.zero_grad()
    inter_criterion_loss.backward()
    optimizer.step()
    count+=1
    if(count%25000==0):
      print(count)
      torch.save(model.state_dict(),str(count)+ str(i)+'layer.pth')
      print('saved')
  print(total_criterion_loss)


  
 
  


25000
saved
50000
saved
75000
saved
100000
saved
125000
saved
150000
saved
175000
saved
200000
saved
225000
saved
250000
saved
275000
saved
300000
saved
325000
saved
350000
saved
375000
saved
400000
saved
425000
saved
450000
saved
475000
saved
500000
saved
tensor(1228426.6250)
25000
saved
50000
saved
75000
saved
100000
saved
125000
saved
150000
saved
175000
saved
200000
saved
225000
saved
250000
saved
275000
saved
300000
saved
325000
saved
350000
saved
375000
saved
400000
saved
425000
saved
450000
saved
475000
saved
500000
saved
tensor(837908.5625)
25000
saved
50000
saved
75000
saved
100000
saved
125000
saved
150000
saved
175000
saved
200000
saved
225000
saved
250000
saved
275000
saved
300000
saved
325000
saved
350000
saved
375000
saved
400000
saved
425000
saved
450000
saved
475000
saved
500000
saved
tensor(734124.1875)
25000
saved
50000
saved
75000
saved
100000
saved
125000
saved
150000
saved
175000
saved
200000
saved
225000
saved
250000
saved
275000
saved
300000
saved
325000
saved
35